In [201]:
import json
from pathlib import Path

TSV_PATH = Path("/Users/joregan/Playing/hsi/annotations/word_annotations/main")
JSON_PATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [202]:
old_json = {}
tsv_cache = {}

In [214]:
def slurp(filename):
    with open(filename) as f:
        segment = json.load(f)
    return segment

In [220]:
def get_topic_context(segment, size=None, keep_topic=True):
    rec_id = segment["recording_id"]
    orig_seg_id = segment["segment_id"]
    if not "recording_id" in old_json:
        with open(JSON_PATH / f"{rec_id}.json") as inf:
            old_json["recording_id"] = json.load(inf)
    original = old_json["recording_id"]
    orig_keys = list(original.keys())
    orig_topic = original[orig_seg_id]["high_level"]["current_topic"]

    index = orig_keys.index(orig_seg_id)
    if size is None:
        start = 0
    else:
        start = index - size
    ctx_range = orig_keys[start:index]

    if size is not None and len(ctx_range) < size:
        if int(orig_seg_id) <= size:
            pass
        else:
            print(f"Warning: size of {size} cannot be satisfied: {ctx_range}")
    
    topics = [original[x]["high_level"]["current_topic"] for x in ctx_range]

    tmp = []
    for p in zip(ctx_range, topics):
        if not keep_topic:
            tmp.append(original[p[0]]["snippet"])
        elif keep_topic and p[1] == orig_topic:
            tmp.append(original[p[0]]["snippet"])
        else:
            tmp.append(None)
    return " ".join([x for x in tmp if x is not None])

In [ ]:
seg = slurp("/Users/joregan/Playing/hsi/annotations/meta/hsi_7_0719_227_003_68_000_meta.json")
txt_ctx = get_topic_context(seg, 5)


{'0': {'general': {'id': 0, 'start': 3.326490706609068, 'end': 3.9640526787882777}, 'high_level': {'topic_name': 'conversation_generic', 'current_topic': 'room', 'topic_change': False, 'topic_duration_id': 0, 'spatial_reference': '', 'referenced_object': []}, 'low_level': {'resolved_references': {}, 'spatial_relationships': []}, 'snippet': 'Yeah.'}, '1': {'general': {'id': 1, 'start': 5.8169671604341, 'end': 7.703087994797592}, 'high_level': {'topic_name': 'conversation_generic', 'current_topic': 'room', 'topic_change': False, 'topic_duration_id': 1, 'spatial_reference': '', 'referenced_object': []}, 'low_level': {'resolved_references': {'it': 'Wall'}, 'spatial_relationships': [], 'resolved_adverbs': []}, 'snippet': "Yeah, it's a really nice place actually."}, '2': {'general': {'id': 2, 'start': 9.007218602708168, 'end': 10.22921238271832}, 'high_level': {'topic_name': 'conversation_generic', 'current_topic': 'room', 'topic_change': False, 'topic_duration_id': 2, 'spatial_reference': '

"Yeah. Ah. That one, it seems like it doesn't have enough light because it's a little bit sad. So I would just put them really, really next to the windows and where the sun comes because you're not facing south here, you're facing east. So yeah, I would put it like next to the do- next to the door and really keep it there in the mornings because um I think it really needs some light."

In [221]:
def get_time_context(segment, ctx_time = 5.0):
    rec_id = segment["recording_id"]
    start = segment["timing"]["utterance_start"]

    if not rec_id in tsv_cache:
        with open(TSV_PATH / f"{rec_id}_main.tsv") as inf:
            lines = []
            for line in inf.readlines():
                line = line.strip()
                if "\t" in line:
                    lines.append(line.split("\t"))
            tsv_cache[rec_id] = lines

    tsv_times = tsv_cache[rec_id]
    extract = []
    for time in tsv_times:
        s = float(time[0])
        e = float(time[1])
        if s >= (start - ctx_time) and (e < start):
            extract.append(time[2])
    return " ".join(extract)

In [197]:
get_time_context(data, "12", 5.0)

'yeah ok ok'